In [1]:
import gwaslab as gl
import pandas as pd

In [15]:
# 1. Load your original UKBB dataset (hg19)
df = pd.read_csv(
    "/mnt/hdd_1/ofgeha/galaxy-gwas-tools/Data/21001_raw.gwas.imputed_v3.both_sexes.tsv.bgz",
    sep="\t",
    compression="gzip")

df[['CHR', 'POS', 'REF', 'ALT']] = df['variant'].str.split(":", expand=True)
df['POS'] = df['POS'].astype(int)

df_standard = df.rename(columns={
    'variant': 'SNPID', 'ALT': 'EA', 'REF': 'NEA',
    'minor_AF': 'EAF', 'beta': 'BETA', 'se': 'SE', 'pval': 'P'})

In [17]:
# 2. Initialize and Fix
mysumstats = gl.Sumstats(df_standard, build="19")
mysumstats.basic_check()

2026/01/10 16:12:58 GWASLab v4.0.4 https://cloufield.github.io/gwaslab/
2026/01/10 16:12:58 (C) 2022-2026, Yunye He, Kamatani Lab, GPL-3.0 license, gwaslab@gmail.com
2026/01/10 16:12:58 Python version: 3.12.3 (main, Nov  6 2025, 13:44:16) [GCC 13.3.0]
2026/01/10 16:12:58 Start to initialize gl.Sumstats from pandas DataFrame ...
2026/01/10 16:12:59  -Reading columns          : 
2026/01/10 16:12:59  -Renaming columns to      : 
2026/01/10 16:12:59  -Current Dataframe shape : 13791467  x  15
2026/01/10 16:13:00  -Initiating a status column: STATUS ...
2026/01/10 16:13:00  -Genomic coordinates are based on GRCh37/hg19...
2026/01/10 16:13:02 Start to reorder the columns ...(v4.0.4)
2026/01/10 16:13:02  -Reordering columns to    : SNPID,CHR,POS,EA,NEA,STATUS,EAF,BETA,SE,P,minor_allele,low_confidence_variant,n_complete_samples,AC,ytx,tstat
2026/01/10 16:13:03 Finished reordering the columns.
2026/01/10 16:13:03  -Trying to convert datatype for CHR: object -> Int64...Failed...
2026/01/10 16:13

,SNPID,CHR,POS,EA,NEA,STATUS,EAF,BETA,SE,P,minor_allele,low_confidence_variant,n_complete_samples,AC,ytx,tstat
0,1:69487:G:A,1,69487,A,G,1960099,0.000006,-2.715450,2.360060,0.249902,A,True,359983,4.15294,103.515,-1.150590
1,1:69569:T:C,1,69569,C,T,1960099,0.000188,-0.484284,0.423462,0.252778,C,True,359983,135.27800,3644.560,-1.143630
2,1:139853:C:T,1,139853,T,C,1960099,0.000006,-2.703560,2.360130,0.251997,T,True,359983,4.09020,101.840,-1.145510
3,1:692794:CA:C,1,692794,C,CA,1960399,0.110590,-0.016436,0.019585,0.401342,C,False,359983,79621.10000,2179000.000,-0.839228
4,1:693731:A:G,1,693731,G,A,1960099,0.115767,-0.004255,0.018507,0.818155,G,False,359983,83348.00000,2281760.000,-0.229918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13779880,X:154929412:C:T,23,154929412,T,C,1960099,0.245473,-0.016260,0.010723,0.129427,T,False,359983,176732.00000,4837480.000,-1.516370
13779881,X:154929637:CT:C,23,154929637,C,CT,1960399,0.229726,-0.027098,0.011190,0.015456,C,False,359983,165395.00000,4525550.000,-2.421510
13779882,X:154929952:CAA:C,23,154929952,C,CAA,1960399,0.239430,-0.020494,0.011278,0.069202,C,False,359983,172381.00000,4717910.000,-1.817100
13779883,X:154930230:A:G,23,154930230,G,A,1960099,0.245887,-0.016347,0.010721,0.127334,G,False,359983,177030.00000,4845680.000,-1.524700


In [18]:
# 3. Liftover hg19 -> hg38
mysumstats.liftover(from_build="19", to_build="38")

2026/01/10 16:14:39 Start to perform liftover ...(v4.0.4)
2026/01/10 16:14:39  -Using built-in chain file: /mnt/hdd_1/ofgeha/galaxy-gwas-tools/.venv/lib/python3.12/site-packages/gwaslab/data/chains/hg19ToHg38.over.chain.gz
2026/01/10 16:14:41  -Converting variants with status code xxx0xxx: 13,779,885
2026/01/10 16:14:41  -Target build: 38
2026/01/10 16:14:41  -Input positions are 1-based
2026/01/10 16:14:41  -Output positions will be 1-based
2026/01/10 16:16:16  -Chromosome mismatches detected: 1896 variants (treated as unmapped)
2026/01/10 16:16:16  -Examples of chromosome mismatches:
2026/01/10 16:16:16    SNPID=1:13123419:T:A | CHR=1 | POS=13123419 | CHR_LIFT=1_KI270766v1_alt | POS_LIFT=1490 | STATUS=1960099
2026/01/10 16:16:16    SNPID=1:13123431:A:C | CHR=1 | POS=13123431 | CHR_LIFT=1_KI270766v1_alt | POS_LIFT=1502 | STATUS=1960099
2026/01/10 16:16:16    SNPID=1:13124908:G:A | CHR=1 | POS=13124908 | CHR_LIFT=1_KI270766v1_alt | POS_LIFT=2979 | STATUS=1960099
2026/01/10 16:16:16    

In [19]:
# 4. Comprehensive Harmonization (The All-in-One way)
# This handles check_ref, flip_allele_stats, and rsID assignment in one pass
mysumstats.harmonize(
    ref_seq="/mnt/hdd_1/ofgeha/gwas-sumstats-harmoniser/gwas-ref/hg38.fa",
    ref_rsid_tsv=gl.get_path("1kg_dbsnp151_hg38_auto"),
    sweep_mode=True,  # Critical for 13M+ variants
    threads=8         # Use more cores if available
)

2026/01/10 16:17:26 Start to check SNPID/rsID ...(v4.0.4)
2026/01/10 16:17:26  -Checking SNPID data type...
2026/01/10 16:17:26  -Checking if SNPID contains NA strings :na,NA,Na,Nan,NaN,<NA>,null,NULL,#N/A,#VALUE!,N/A,n/a,missing,...
2026/01/10 16:17:27  -Checking if SNPID is CHR:POS:NEA:EA...(separator: - ,: , _)
2026/01/10 16:17:37 Finished checking SNPID/rsID.
2026/01/10 16:17:37 Start to fix chromosome notation (CHR) ...(v4.0.4)
2026/01/10 16:17:37  -Checking CHR data type...
2026/01/10 16:17:41  -Variants with standardized chromosome notation: 13771094
2026/01/10 16:17:41  -All CHR are already fixed...
2026/01/10 16:17:44 Finished fixing chromosome notation (CHR).
2026/01/10 16:17:44 Start to fix basepair positions (POS) ...(v4.0.4)
2026/01/10 16:17:44  -Trying to convert datatype for POS: Int64 -> Int64...
2026/01/10 16:17:45  -Position bound:(0 , 250,000,000)
2026/01/10 16:17:45  -No outlier variants were removed.
2026/01/10 16:17:46 Finished fixing basepair positions (POS).
202

,SNPID,rsID,CHR,POS,EA,NEA,STATUS,EAF,BETA,SE,P,minor_allele,low_confidence_variant,n_complete_samples,AC,ytx,tstat
0,1:69487,<NA>,1,69487,A,G,3860099,0.000006,-2.715450,2.360060,0.249902,A,True,359983,4.15294,103.515,-1.150590
1,1:69569,<NA>,1,69569,C,T,3860099,0.000188,-0.484284,0.423462,0.252778,C,True,359983,135.27800,3644.560,-1.143630
2,1:139853,<NA>,1,139853,T,C,3860099,0.000006,-2.703560,2.360130,0.251997,T,True,359983,4.09020,101.840,-1.145510
3,1:757414,<NA>,1,757414,C,CA,3860399,0.110590,-0.016436,0.019585,0.401342,C,False,359983,79621.10000,2179000.000,-0.839228
4,1:758351,<NA>,1,758351,G,A,3860099,0.115767,-0.004255,0.018507,0.818155,G,False,359983,83348.00000,2281760.000,-0.229918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13771089,23:155699751,<NA>,23,155699751,T,C,3860099,0.245473,-0.016260,0.010723,0.129427,T,False,359983,176732.00000,4837480.000,-1.516370
13771090,23:155699976,<NA>,23,155699976,C,CT,3860399,0.229726,-0.027098,0.011190,0.015456,C,False,359983,165395.00000,4525550.000,-2.421510
13771091,23:155700291,<NA>,23,155700291,C,CAA,3860399,0.239430,-0.020494,0.011278,0.069202,C,False,359983,172381.00000,4717910.000,-1.817100
13771092,23:155700569,<NA>,23,155700569,G,A,3860099,0.245887,-0.016347,0.010721,0.127334,G,False,359983,177030.00000,4845680.000,-1.524700


In [21]:
# 5. Optional: Infer Strand (Only if you have palindromic SNPs A/T or G/C)
# Use infer_strand2 for speed on large datasets
mysumstats.infer_strand2(
    vcf_path=gl.get_path("1kg_eur_hg38"), # Use appropriate population ref
    maf_threshold=0.40,
    threads=8
)
mysumstats.flip_allele_stats() # Apply the strand flips

2026/01/10 16:26:42 No records in config file. Please download first.
2026/01/10 16:26:42 Start to annotate and infer strand orientation using allele frequencies ...(v4.0.4)
2026/01/10 16:26:42  -Number of threads/cores to use: 8


AttributeError: 'bool' object has no attribute 'lower'

In [22]:
# 6. Final Liftover to T2T (CHM13)
mysumstats.liftover(
    from_build="38", 
    to_build="13", 
    chain_path="./grch38-chm13v2.chain"
)

2026/01/10 16:26:50 Start to perform liftover ...(v4.0.4)
2026/01/10 16:26:50  -Using provided chain file: ./grch38-chm13v2.chain
2026/01/10 16:26:52  -Converting variants with status code xxx0xxx: 13,771,094
2026/01/10 16:26:52  -Target build: 13
2026/01/10 16:26:52  -Input positions are 1-based
2026/01/10 16:26:52  -Output positions will be 1-based
2026/01/10 16:28:24  -Mapped: 13733999 variants
2026/01/10 16:28:24  -Unmapped: 37095 variants
2026/01/10 16:28:24  -Examples of unmapped variants:
2026/01/10 16:28:24    SNPID=1:69487 | CHR=1 | POS=69487 | STATUS=3860099
2026/01/10 16:28:24    SNPID=1:69569 | CHR=1 | POS=69569 | STATUS=3860099
2026/01/10 16:28:24    SNPID=1:139853 | CHR=1 | POS=139853 | STATUS=3860099
2026/01/10 16:28:24    SNPID=1:893791 | CHR=1 | POS=893791 | STATUS=3860399
2026/01/10 16:28:24    SNPID=1:909096 | CHR=1 | POS=909096 | STATUS=3860099
2026/01/10 16:28:33  -Removed 37095 unmapped variants
2026/01/10 16:28:33 Start to fix chromosome notation (CHR) ...(v4.0.4